In [ ]:
import numpy as np
from zampy.datasets import EthCanopyHeight
from zampy.datasets.dataset_protocol import TimeBounds, SpatialBounds
from pathlib import Path

download_dir = Path("/home/bart/Zampy/download")
ingest_dir = Path("/home/bart/Zampy/ingest")
times = TimeBounds(np.datetime64("2020-01-01"), np.datetime64("2020-12-31"))
bbox_demo = SpatialBounds(54, 6, 51, 3)

In [ ]:
canopy_height_dataset = EthCanopyHeight()
canopy_height_dataset.download(
    download_dir=download_dir,
    time_bounds=times,
    spatial_bounds=bbox_demo,
    variable_names=["canopy-height"],
)

In [ ]:
canopy_height_dataset.ingest(download_dir, ingest_dir)

In [ ]:
ds = canopy_height_dataset.load(
    ingest_dir=ingest_dir,
    time_bounds=times,
    spatial_bounds=bbox_demo,
    variable_names=["canopy-height"],
)

In [ ]:
ds